# Subgraphs


In [1]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from  langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
parent_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
subgraph_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")  # can be different model

In [2]:
class ParentState(TypedDict):  # final output structure

    question: str
    answer_eng: str
    answer_hin: str

In [3]:
def translate_to_hindi(state : ParentState):
    prompt = f"""
        Translate the following text to hindi
        Keep it natural and clear , Do not add extra content
        Text : 
            {state["answer_eng"]}
    """.strip()
    
    translated_text = subgraph_llm.invoke(prompt).content
    return {"answer_hin" : translated_text}


**SubGraph**

In [4]:
# Build SubGraph
subgraph_builder = StateGraph(ParentState)
subgraph_builder.add_node("translate_text" , translate_to_hindi)

subgraph_builder.add_edge(START , "translate_text")
subgraph_builder.add_edge("translate_text" , END)

subgraph = subgraph_builder.compile()


**Parent Graph**

In [5]:
def generate_answer(state: ParentState):

    answer = parent_llm.invoke(f"You are a helpful assistant. Answer clearly.\n\nQuestion: {state['question']}").content
    return {'answer_eng': answer}

In [6]:
parent_builder = StateGraph(ParentState)

parent_builder.add_node("answer", generate_answer)
parent_builder.add_node("translate", subgraph)

parent_builder.add_edge(START, 'answer')
parent_builder.add_edge('answer', 'translate')
parent_builder.add_edge('translate', END)
graph = parent_builder.compile()

In [7]:
graph.invoke({'question': 'What is quantum physics'})

{'question': 'What is quantum physics',
 'answer_eng': 'Quantum physics (also known as quantum mechanics or quantum theory) is a **branch of physics that studies matter and energy at the most fundamental level.**\n\nIt explores the **bizarre and counter-intuitive rules that govern the universe at its smallest scales,** where the familiar laws of classical physics (like Newton\'s laws of motion) break down.\n\nHere\'s a breakdown of what that means:\n\n1.  **The World of the Very Small:** Quantum physics deals with the behavior of atoms and subatomic particles (like electrons, protons, neutrons, and photons). At this tiny scale, things behave in ways that are radically different from the macroscopic world we experience every day.\n\n2.  **Energy Comes in "Packets" (Quanta):** One of the foundational ideas is that energy is not continuous but comes in discrete, indivisible packets called "quanta." Think of it like a staircase instead of a ramp – you can only be on specific steps, not in 